In [2]:
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from sklearn.cluster import DBSCAN
import simplekml
from smount_predictors import SeamountHelp
import xarray as xr

In [5]:
training_mounts = pd.read_csv('out/train_mounts.csv')
points = SeamountHelp.readKMLbounds(Path('data/seamount_training_zone.kml'))
testing_data = SeamountHelp.readAndFilterGRD(Path('data/vgg_swot.grd'), points[:2], points[2:]).to_dataframe().reset_index()
labeled_data = SeamountHelp.seamount_radial_match(testing_data, training_mounts)
px.scatter(labeled_data, x='lon', y='lat', color='Labels').show()

In [7]:
training_set = xr.Dataset.from_dataframe(labeled_data)
training_set.to_netcdf('out/training_set.nc')

<xarray.Dataset> Size: 746kB
Dimensions:  (index: 20736)
Coordinates:
  * index    (index) int64 166kB 0 1 2 3 4 5 ... 20731 20732 20733 20734 20735
Data variables:
    lat      (index) float64 166kB -19.24 -19.23 -19.21 ... -16.89 -16.88 -16.86
    lon      (index) float64 166kB -112.9 -112.9 -112.9 ... -110.5 -110.5 -110.5
    z        (index) float32 83kB 1.217 2.48 2.897 ... 0.3645 -0.2757 2.758
    Labels   (index) int64 166kB 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0

In [7]:
aall = pd.read_csv('data/all.xyhrdnc', sep=' ', names=[
    'lon', 'lat', 'height', 'radius', 'depth', 'name', 'catalouged'
])
import simplekml
kml = simplekml.Kml()
for i, row in aall.iterrows():
    pnt = kml.newpoint(name=row['name'], coords=[(row['lon'], row['lat'])])
    pnt.description = f'Height: {row["height"]}\nRadius: {row["radius"]}\nDepth: {row["depth"]}\nCatalouged: {row["catalouged"]}'
kml.save('data/all_cat.kml')

In [9]:
! open data/all_cat.kml

## TODO: fix testing data - catalouge does not contain many small seamounts